In [1]:
# !pip install crewai crewai_tools langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew, LLM

In [4]:
choose_LLM = 1
colab_exp = 0
import os

from langchain_openai import ChatOpenAI

if choose_LLM==1:
    if(colab_exp):
        from google.colab import userdata    ###for colab use
        llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=userdata.get('OPENAI_API_KEY'))
    else:
        from dotenv import load_dotenv
        load_dotenv() 
        llm = "gpt-4o-mini" #ChatOpenAI(model = "gpt-4o-mini",openai_api_key=os.getenv("OPENAI_API_KEY"))

else:
    llm = ChatOpenAI(model="ollama/llama3.2", base_url="http://localhost:11434",temperature=0.1)
    


In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm = llm,
	verbose=True
)

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
 	allow_delegation= True,
    llm = llm,
)

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, \
                         WebsiteSearchTool
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/crews"
)

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=1,
  memory=True
)

In [11]:
inputs = {
    "customer": "ResolveAI",
    "person": "Usman",
    "inquiry": "I need help with setting up a CrewAI for safe city "
               "how can I add memory and tools to my agentic system? "
               "Can you provide guidance with the steps?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: ResolveAI just reached out with a super important ask:
I need help with setting up a CrewAI for safe city how can I add memory and tools to my agentic system? Can you provide guidance with the steps?

Usman from ResolveAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: I need to gather detailed information about setting up a CrewAI for a safe city, particularly focusing on adding memory and tools to the agentic system. I will refer to the content from the CrewAI documentation to assist Usman from ResolveAI effectively.
## Using tool: [Read website content]
## Tool Input: 
"{}"
## Tool Output: 
Crews - CrewAISkip to main contentCrewAI home pageEnglishSearch...⌘KStart Cloud TrialcrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationCore ConceptsCrewsHo

In [12]:
from IPython.display import display, Markdown
Markdown(result.raw)

To set up a CrewAI for a safe city and add memory and tools to your agentic system, you can follow these steps:

1. **Define Your Crew**: Use a class that inherits from `CrewBase` and employ decorators to manage agents and tasks. Here is a simple example to illustrate this:
   ```python
   from crewai import CrewBase, agent, task, crew

   @CrewBase
   class SafeCityCrew:
       @agent
       def city_monitor_agent(self):
           return Agent(role="City Monitor", goal="Monitor city activity.")

       @task
       def analyze_traffic_task(self):
           return Task(description="Analyze traffic data.")

       @crew
       def crew(self):
           return Crew(agents=[self.city_monitor_agent()], tasks=[self.analyze_traffic_task()])
   ```

2. **Adding Memory**: You can utilize the `memory` attribute in the crew definition. This can manage short-term, long-term, and entity memory, which enhances decision-making and execution strategies. For example:
   ```python
   @crew
   def crew(self):
       return Crew(
           agents=[self.city_monitor_agent()],
           tasks=[self.analyze_traffic_task()],
           memory={"short_term": {}, "long_term": {}}
       )
   ```

3. **Integrate Tools**: Make sure to define tools that your agents can utilize. You can integrate custom tools or built-in ones. Here’s an example of how to define a tool:
   ```python
   from crewai_tools import SomeCustomTool

   @agent
   def traffic_analysis_agent(self):
       return Agent(
           role="Traffic Analyst",
           tools=[SomeCustomTool()]
       )
   ```

4. **Kickoff the Crew Execution**: Use the `kickoff()` function to start your execution process as structured:
   ```python
   my_crew = SafeCityCrew().crew()
   result = my_crew.kickoff(inputs={})
   print(result)
   ```

5. **Monitor Output and Logs**: It's important to track the logs and outputs to gain insights from the execution for further improvements. You can do this by setting `output_log_file=True` during the execution to save logs.

These steps provide a structured approach to building your agentic system using CrewAI. If you have any specific requirements or additional functionalities in mind, please don’t hesitate to share them. I would be more than happy to assist you further with your project! 

We are here to help, and that means addressing any further inquiries you might have! 🚀